In [1]:
from datetime import datetime

import pandas as pd

from data.get_data import get_data_from_collection

2022-05-25 14:06:42.078 WARNING streamlit.state.auto_session_state: Session state does not function when running a script without `streamlit run`


Unable to connect to the server.


In [132]:
select_rows = {
    "date": 1,
    "name": 1,
    "eventAction": 1,
    "hour": 1,
    "totalEvents": 1,
    "uniqueEvents": 1,
    "eventValue": 1,
    "_id": 0,
}
query = {}

agg = {"totalEvents": "sum", "uniqueEvents": "sum", "eventValue": "sum"}
df = get_data_from_collection("video", query, select_rows)
df["day"] = df["date"].dt.isocalendar().day
df["week"] = df["date"].dt.isocalendar().week
df["month"] = df["date"].dt.month
df

,eventAction,date,hour,totalEvents,uniqueEvents,eventValue,name,day,week,month
0,durationchange,2022-03-06,17,1,1,0,Lost in Taste (S1E1),7,9,3
1,durationchange,2022-03-06,17,1,0,389,Lost in Taste (S1E1),7,9,3
2,durationchange,2022-03-07,12,1,1,33,Mix (S3E3),1,10,3
3,durationchange,2022-03-07,9,1,0,10,Mix (S3E3),1,10,3
4,durationchange,2022-03-07,10,1,1,74,Cardi's WhipShots (S1E1),1,10,3
...,...,...,...,...,...,...,...,...,...,...
64777,waiting,2022-01-31,16,1,0,487,American Whiskey: W. B. Saffell & Bond and Lil...,1,5,1
64778,waiting,2022-01-31,7,1,1,394,NaN,1,5,1
64779,waiting,2022-02-01,14,1,1,0,World Whisky: Puni Arte Limited Edition 01 & M...,2,5,2
64780,waiting,2022-02-01,5,1,1,0,Home Bar Hero (S2E3),2,5,2


In [133]:
def top_events(data, event, start_date, end_date, top_n = 10):
    agg = {"totalEvents": "sum", "uniqueEvents": "sum", "eventValue": "sum"}
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    df = data.copy()
    df["date"] = pd.to_datetime(df["date"])
    # greater than the start date and smaller than the end date
    df = df.loc[df["date"].between(start_date, end_date)]
    # df = df[df['name'].str.startswith('#')]
    df = df[df["eventAction"] == event].groupby(["name", "date"]).agg(agg).reset_index()
    df = df.sort_values(by=["eventValue"], ascending=False).reset_index(drop=True)
    return df.iloc[:top_n, :]

top_events(df, "ended", "2020-01-01", "2022-12-31")

,name,date,totalEvents,uniqueEvents,eventValue
0,Cocktail History (S1E1),2022-01-20,12,12,7884
1,#WhiskyWednesdays (S1E2),2022-02-13,2,1,4708
2,Cardi's WhipShots (S1E1),2022-01-21,5,5,4435
3,Wine Uncorked (S1E1),2022-02-25,2,2,3652
4,Cardi's WhipShots (S1E1),2022-02-15,4,4,3548
5,Cardi's WhipShots (S1E1),2022-02-03,4,4,3548
6,Cardi's WhipShots (S1E2),2022-01-01,4,4,2872
7,Cardi's WhipShots (S1E2),2022-02-18,4,4,2872
8,Cardi's WhipShots (S1E2),2022-02-11,4,4,2872
9,Cardi's WhipShots (S1E2),2022-02-07,4,4,2872


In [119]:
df['eventAction'].unique()

array(['durationchange', 'pause', 'playing', 'progress', 'ended',
       'seeked', 'seeking', 'volumechange', 'waiting'], dtype=object)

In [128]:
df = df[df['name'].str.startswith('#')]


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [131]:
df['name'].str.startswith('#')

0        False
1        False
2        False
3        False
4        False
         ...  
64777    False
64778      NaN
64779    False
64780    False
64781    False
Name: name, Length: 64782, dtype: object

In [97]:
df['date']
mask = (df['dates'] > start_date) & (df['dates'] <= end_date)


0       2022-03-06
1       2022-03-06
2       2022-03-07
3       2022-03-07
4       2022-03-07
           ...    
64777   2022-01-31
64778   2022-01-31
64779   2022-02-01
64780   2022-02-01
64781   2022-01-31
Name: date, Length: 64782, dtype: datetime64[ns]

In [99]:
mask = (df['date'] > start_date) & (df['date'] <= end_date)

df.loc[mask]

,eventAction,date,hour,totalEvents,uniqueEvents,eventValue,name,day,week,month


In [109]:

df.loc[df['date'] > start_date]

,eventAction,date,hour,totalEvents,uniqueEvents,eventValue,name,day,week,month
0,durationchange,2022-03-06,17,1,1,0,Lost in Taste (S1E1),7,9,3
1,durationchange,2022-03-06,17,1,0,389,Lost in Taste (S1E1),7,9,3
2,durationchange,2022-03-07,12,1,1,33,Mix (S3E3),1,10,3
3,durationchange,2022-03-07,9,1,0,10,Mix (S3E3),1,10,3
4,durationchange,2022-03-07,10,1,1,74,Cardi's WhipShots (S1E1),1,10,3
...,...,...,...,...,...,...,...,...,...,...
64777,waiting,2022-01-31,16,1,0,487,American Whiskey: W. B. Saffell & Bond and Lil...,1,5,1
64778,waiting,2022-01-31,7,1,1,394,NaN,1,5,1
64779,waiting,2022-02-01,14,1,1,0,World Whisky: Puni Arte Limited Edition 01 & M...,2,5,2
64780,waiting,2022-02-01,5,1,1,0,Home Bar Hero (S2E3),2,5,2
